# Web scraping - transfers/contracts

In [1]:
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import requests
import pandas as pd
import requests

In [11]:
load_dotenv()

def get_soup(url):
    page = requests.get(url,headers = {"User-Agent":"Mozilla/5.0","origin":"https://www.transfermarkt.co.uk"})
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [14]:
transfer_soup = get_soup('https://www.transfermarkt.co.uk/premier-league/transfers/wettbewerb/GB1?saison_id=2020')
main_div = transfer_soup.find('div',{'class':'large-8 columns'})


<div class="large-8 columns">
<div class="box">
<div class="table-header">
				Transfers 20/21				<div class="header-social">
<div class="teilen-desktop-link" name="teilen-desktop" onclick="tmEvent('teilen-desktop','click','teilen');
">
<ul>
<li><a class="MailToLink" href="mailto:?to=&amp;subject=Transfers&amp;body=https%3A%2F%2Fwww.transfermarkt.co.uk%2Fpremier-league%2Ftransfers%2Fwettbewerb%2FGB1%3Fsaison_id%3D2020" name="teilen-mail-desktop" onclick="tmEvent('teilen-mail-desktop','click','teilen');
"><img alt="Mail" src="https://tmsi.akamaized.net/icons/mail-schwarz.svg"/></a></li>
<li><a class="TwitterLink" href="https://twitter.com/share?url=https%3A%2F%2Fwww.transfermarkt.co.uk%2Fpremier-league%2Ftransfers%2Fwettbewerb%2FGB1%3Fsaison_id%3D2020&amp;text=Transfers" name="teilen-twitter-desktop" onclick="tmEvent('teilen-twitter-desktop','click','teilen');
" target="_blank"><img alt="Twitter" src="https://tmsi.akamaized.net/icons/Twitter-Blau.svg"/></a></li>
<li><a class="FacebookLi

In [48]:
player_urls = []
for team in main_div.find_all('div',{'class':'box'})[3:]:
    team_name = team.find('div',{'class':'table-header'}).find('h2')
    transfers = team.find_all('div',{'class':'responsive-table'})
    transfer_table = transfers[0].find('tbody')
    for transfer_table in transfers[0:1]:
        transfer_table = transfer_table.find('tbody')
        for tr in transfer_table.find_all('tr'):
            td = tr.find('td')
            span = td.find('div').find('span')
            link = span.find('a', {'class':'spielprofil_tooltip'})
            player_urls.append([link.text,link["href"]])
                  

[['Thomas Partey', '/thomas-partey/profil/spieler/230784'], ['Gabriel', '/gabriel/profil/spieler/435338'], ['Pablo Marí', '/pablo-mari/profil/spieler/210178'], ['Martin Ødegaard', '/martin-odegaard/profil/spieler/316264'], ['Rúnar Alex Rúnarsson', '/runar-alex-runarsson/profil/spieler/205657'], ['Cédric Soares', '/cedric-soares/profil/spieler/112988'], ['Willian', '/willian/profil/spieler/52769'], ['Mathew Ryan', '/mathew-ryan/profil/spieler/128969'], ['Dejan Iliev', '/dejan-iliev/profil/spieler/205757'], ['Joe Willock', '/joe-willock/profil/spieler/340329'], ['Emile Smith Rowe', '/emile-smith-rowe/profil/spieler/392765'], ['William Saliba', '/william-saliba/profil/spieler/495666'], ['Mohamed Elneny', '/mohamed-elneny/profil/spieler/160438'], ['Henrikh Mkhitaryan', '/henrikh-mkhitaryan/profil/spieler/55735'], ['Dejan Iliev', '/dejan-iliev/profil/spieler/205757'], ['Ollie Watkins', '/ollie-watkins/profil/spieler/324358'], ['Bertrand Traoré', '/bertrand-traore/profil/spieler/131996'], ['

In [55]:

def get_player_transfers(player):
    soup_url = player[1].replace('/profil/','/transfers/')
    player_soup = get_soup('https://www.transfermarkt.co.uk'+soup_url)
    transfer_history = player_soup.find_all('tr', {'class':'zeile-transfer'})
    transfer_list = []
    for tr in transfer_history:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td if tr.text.strip() != '']
        row.append(player[0])
        transfer_list.append(row)
        
    df = pd.DataFrame(transfer_list, columns = ["season","date","left","joined","mv","fee","name"])
    print(player + ' added')
    return df

In [56]:
import time
import random

df = pd.DataFrame([], columns = ["season","date","left","joined","mv","fee","name"])
for player in player_urls:
    player_df = get_player_transfers(player)
    df = pd.concat([df,player_df])
    time.sleep(random.randint(15, 26))
df

,season,date,left,joined,mv,fee,name
0,20/21,"Oct 5, 2020",Atlético Madrid,Arsenal,£36.00m,£45.00m,Thomas Partey
1,15/16,"Jul 1, 2015",Atl. Madrid B,Atlético Madrid,£3.60m,-,Thomas Partey
2,14/15,"Jun 30, 2015",UD Almería,Atl. Madrid B,£2.25m,End of loan,Thomas Partey
3,14/15,"Jul 27, 2014",Atl. Madrid B,UD Almería,£900Th.,loan transfer,Thomas Partey
4,13/14,"Jun 30, 2014",RCD Mallorca,Atl. Madrid B,£900Th.,End of loan,Thomas Partey
...,...,...,...,...,...,...,...
5,11/12,"Feb 1, 2012",Norwich,Peterborough,£225Th.,loan transfer,Ryan Bennett
6,11/12,"Jan 1, 2012",Peterborough,Norwich,£225Th.,£3.47m,Ryan Bennett
7,09/10,"Oct 1, 2009",Grimsby Town,Peterborough,-,£608Th.,Ryan Bennett
8,07/08,"Jul 1, 2007",Grimsby U18,Grimsby Town,-,-,Ryan Bennett


In [59]:
df.to_csv('./CSVs/transfers_2.csv')